# Importing Packages and Loading the KfW Project Dataset

In this section, we will import the necessary packages and load the KfW project dataset from a JSON file.

In [46]:
# Reading the KfW project database and using a 10-topic LDA to label the projects:

import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
from collections import Counter

# Define file paths
json_file_path = 'files/projects.json'


# Import necessary packages to read the KFW project dataset
import json
import pandas as pd

# Load json file
with open("files/projects.json", encoding="utf-8") as file:
    data = json.load(file)

# Convert to pandas DataFrame
df = pd.json_normalize(data)

# Show the first few rows of the DataFrame
df.head()



,projnr,title,description,amount,country,focus,currency,responsible,principal,crscode2,crscode5,finanzierungsinstrument,status,hostDate,fzRegion,projekttraegers,kofinanzpartners,usvkategorie,usvbeschr
0,5060,KV-Abwasserentsorgung Salfeet,Im Rahmen des Vorhabens soll in der Stadt Salf...,19.393,Palästinensische Gebiete,Wasser und Abfall,EUR,Nahost,BMZ,140 - Wasser und Abwasser-/Abfallentsorgung,"14020 - Wasser-, Sanitärver. und Abwassermanag...",Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,[MUNICIPALITY OF SALFEET],[],B,
1,52072,Regionaler Energieeffizienzfonds westlicher Ba...,Um dem Klimawandel entgegenzuwirken und eine n...,20.000,Südosteuropa,Wirtschaft und Finanzsektor,EUR,Beteiligungsfinanzierung,BMZ,240 - Finanzwesen,24030 - Finanzintermediäre des formellen Sektors,Treuhandbeteiligung,aktiv,"Jul 31, 2023",Europa/Kaukasus,"[GREEN FOR GROWTH FUND, SOUTHEAST EUROPE SA, S...",[],FI/A,Das Vorhaben wurde in die Umwelt- und Sozialri...
2,52087,"""Prep4Vac"" COVID-19 Gesundheitsvorhaben mit de...",Im Kontext der COVID-19 Pandemie hat sich die ...,12.220,Afrikanische Union (AU=African Union),Gesundheit,EUR,Ostafrika und Afrikanische Union,BMZ,250 - Privatwirtschaftliche und andere Dienste,25030 - Geschäftsentwicklung,Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Afrika Subsahara,[AFRICAN EXPORT-IMPORT BANK],[],FI/B,Die Umwelt- und Sozialverträglichkeit wird für...
3,52442,Investition in tunesische Gesundheitskapazität...,Die in Tunesien seit Ende 2020 stark angestieg...,10.000,Tunesien,Sonstige,EUR,NfR/Beteiligungen/MENA,BMZ,120 - Gesundheitswesen,12264 - COVID-19-Bekämpfung,Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,[UNITED NATIONS CHILDREN'S FUND],[],B,Das Projekt wurde gemäß der KfW-Nachhaltigkeit...
4,15891,Abwasserentsorgung Gaza-Mitte,Das Vorhaben umfasst nach der Änderungsprüfung...,87.561,Palästinensische Gebiete,Wasser und Abfall,EUR,Nahost,BMZ,140 - Wasser und Abwasser-/Abfallentsorgung,"14020 - Wasser-, Sanitärver. und Abwassermanag...",Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,[COASTAL MUNICIPAL WATER UTILITY],[Eigenbeitrag d. Landes/Own contribution of co...,B,


# Data Preparation and NLP Preprocessing

In this section, we will perform data preparation and preprocessing steps for natural language processing (NLP) on the KFW project dataset.
 
## Tokenization and Preprocessing

Before building the LDA model, we need to preprocess the text data. This involves tokenization, lowercase conversion, removing non-words, removing stopwords, and lemmatization. 

In [24]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')  
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models

# Preparing the text data
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase conversion
    tokens = [token for token in tokens if token.isalpha()]  # Removing non-words
    tokens = [token for token in tokens if token not in stopwords.words("german")]  # Removing stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatization
    return tokens

# Preprocessing the descriptions
df["description_tokens"] = df["description"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/repl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/repl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# LDA Topic Modeling with Gensim

In this section, we will perform Latent Dirichlet Allocation (LDA) topic modeling using the Gensim package. LDA is a popular technique for discovering topics in a collection of documents.

The steps to perform LDA topic modeling with Gensim are as follows:

~~1. Tokenization and Preprocessing~~ (Already Performed)
2. Creating a Dictionary and Corpus
3. Building the LDA Model
4. Evaluating the Model
5. Visualizing the Topics

Let's dive into each step in detail.

In [25]:

# Creating the Corpus
dictionary = corpora.Dictionary(df["description_tokens"])
corpus = [dictionary.doc2bow(tokens) for tokens in df["description_tokens"]]

df["description_tokens"].head()

0    [rahmen, vorhabens, stadt, salfeet, gesundheit...
1    [klimawandel, entgegenzuwirken, nachhaltige, e...
2    [kontext, pandemie, afrikanische, union, au, z...
3    [tunesien, seit, ende, stark, angestiegenen, e...
4    [vorhaben, umfasst, änderungsprüfung, zwei, in...
Name: description_tokens, dtype: object

In [26]:
# Training the LDA Model (Latent Dirichlet Allocation)
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Extracting the dominant topics for each document
df["topic"] = df["description_tokens"].apply(lambda tokens: max(lda_model[dictionary.doc2bow(tokens)], key=lambda x: x[1])[0] + 1)

# Saving the result to the CSV file
df.to_csv("projects_with_topics.csv", index=False)

In [45]:
# Import pandas library
import pandas as pd

# Read the CSV file
projects_topics = pd.read_csv('projects_with_topics.csv')

# Display the first few rows
projects_topics.head()

,projnr,title,description,amount,country,focus,currency,responsible,principal,crscode2,crscode5,finanzierungsinstrument,status,hostDate,fzRegion,projekttraegers,kofinanzpartners,usvkategorie,usvbeschr,description_tokens,topic
0,5060,KV-Abwasserentsorgung Salfeet,Im Rahmen des Vorhabens soll in der Stadt Salf...,19.393,Palästinensische Gebiete,Wasser und Abfall,EUR,Nahost,BMZ,140 - Wasser und Abwasser-/Abfallentsorgung,"14020 - Wasser-, Sanitärver. und Abwassermanag...",Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,['MUNICIPALITY OF SALFEET'],[],B,NaN,"['rahmen', 'vorhabens', 'stadt', 'salfeet', 'g...",5
1,52072,Regionaler Energieeffizienzfonds westlicher Ba...,Um dem Klimawandel entgegenzuwirken und eine n...,20.000,Südosteuropa,Wirtschaft und Finanzsektor,EUR,Beteiligungsfinanzierung,BMZ,240 - Finanzwesen,24030 - Finanzintermediäre des formellen Sektors,Treuhandbeteiligung,aktiv,"Jul 31, 2023",Europa/Kaukasus,"['GREEN FOR GROWTH FUND, SOUTHEAST EUROPE SA, ...",[],FI/A,Das Vorhaben wurde in die Umwelt- und Sozialri...,"['klimawandel', 'entgegenzuwirken', 'nachhalti...",4
2,52087,"""Prep4Vac"" COVID-19 Gesundheitsvorhaben mit de...",Im Kontext der COVID-19 Pandemie hat sich die ...,12.220,Afrikanische Union (AU=African Union),Gesundheit,EUR,Ostafrika und Afrikanische Union,BMZ,250 - Privatwirtschaftliche und andere Dienste,25030 - Geschäftsentwicklung,Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Afrika Subsahara,['AFRICAN EXPORT-IMPORT BANK'],[],FI/B,Die Umwelt- und Sozialverträglichkeit wird für...,"['kontext', 'pandemie', 'afrikanische', 'union...",9
3,52442,Investition in tunesische Gesundheitskapazität...,Die in Tunesien seit Ende 2020 stark angestieg...,10.000,Tunesien,Sonstige,EUR,NfR/Beteiligungen/MENA,BMZ,120 - Gesundheitswesen,12264 - COVID-19-Bekämpfung,Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,"[""UNITED NATIONS CHILDREN'S FUND""]",[],B,Das Projekt wurde gemäß der KfW-Nachhaltigkeit...,"['tunesien', 'seit', 'ende', 'stark', 'angesti...",2
4,15891,Abwasserentsorgung Gaza-Mitte,Das Vorhaben umfasst nach der Änderungsprüfung...,87.561,Palästinensische Gebiete,Wasser und Abfall,EUR,Nahost,BMZ,140 - Wasser und Abwasser-/Abfallentsorgung,"14020 - Wasser-, Sanitärver. und Abwassermanag...",Zuschuss / Darlehen aus Haushaltsmitteln,aktiv,"Jul 31, 2023",Nordafrika/Naher Osten,['COASTAL MUNICIPAL WATER UTILITY'],['Eigenbeitrag d. Landes/Own contribution of c...,B,NaN,"['vorhaben', 'umfasst', 'änderungsprüfung', 'z...",9


In [27]:
# Frequency table of the ten topics

topic_counts = df['topic'].value_counts().sort_index()
topic_counts


1     141
2     592
3      29
4     644
5     107
6     124
7     150
8     243
9     260
10     42
Name: topic, dtype: int64

In [40]:
# Get the top 10 common tokens for each topic

common_tokens = {}

for topic_id in range(1, num_topics + 1):
    topic_tokens = lda_model.get_topic_terms(topic_id-1, topn=10)
    tokens = [dictionary[id] for id, _ in topic_tokens]
    common_tokens[topic_id] = tokens

common_tokens

{1: ['mio',
  'sowie',
  'eur',
  'vorhabens',
  'vorhaben',
  'zielgruppe',
  'bevölkerung',
  'zugang',
  'beitrag',
  'phase'],
 2: ['mio',
  'eur',
  'sowie',
  'phase',
  'rahmen',
  'beitrag',
  'ziel',
  'vorhaben',
  'unterstützung',
  'bevölkerung'],
 3: ['bevölkerung',
  'sowie',
  'vorhaben',
  'mio',
  'zielgruppe',
  'verbesserung',
  'beitrag',
  'eur',
  'umsetzung',
  'finanziert'],
 4: ['fonds',
  'sowie',
  'vorhaben',
  'maßnahmen',
  'zielgruppe',
  'ziel',
  'bevölkerung',
  'stärkung',
  'beitrag',
  'sanad'],
 5: ['vorhaben',
  'sowie',
  'vorhabens',
  'flüchtlinge',
  'beitrag',
  'ziel',
  'verbesserung',
  'maßnahmen',
  'mio',
  'zielgruppe'],
 6: ['sowie',
  'fonds',
  'mio',
  'vorhaben',
  'ziel',
  'rahmen',
  'sollen',
  'eur',
  'unternehmen',
  'beitrag'],
 7: ['mio',
  'eur',
  'sowie',
  'vorhabens',
  'bevölkerung',
  'vorhaben',
  'phase',
  'maßnahmen',
  'verbesserung',
  'sollen'],
 8: ['sowie',
  'mio',
  'vorhaben',
  'ziel',
  'beitrag',
  '

## Hyperparameter Tuning Plan

1. Define a list of hyperparameters to tune
2. Create a function to train and evaluate an LDA model with given hyperparameters
3. Define a grid of hyperparameter combinations
4. Iterate over the grid and train/evaluate LDA models
5. Select the best performing model based on evaluation metrics
6. Train the final LDA model with the selected hyperparameters

Let's start by defining the hyperparameters to tune.

In [29]:
# Define hyperparameters to tune
hyperparameters = {
    'num_topics': [5, 10, 15],
    'passes': [10, 15, 20],
    'alpha': ['auto', 'symmetric', 'asymmetric'],
    'eta': [0.01, 0.1, 0.5]
}


In [9]:
## Function to Train and Evaluate LDA Model

import gensim


def train_evaluate_lda_model(corpus, dictionary, num_topics, passes, alpha, eta):
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        passes=passes,
        alpha=alpha,
        eta=eta
    )
    coherence_model = gensim.models.CoherenceModel(
        model=lda_model,
        texts=corpus,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    return lda_model, coherence_score


In [10]:
## Define a Grid of Hyperparameter Combinations to iterate over:

import itertools

hyperparameters = {
    'num_topics': [5, 10, 15],
    'passes': [10, 15, 20],
    'alpha': ['auto', 'symmetric', 'asymmetric'],
    'eta': [0.01, 0.1, 0.5]
}

hyperparameter_combinations = list(itertools.product(*hyperparameters.values()))


In [10]:
## Train and Evaluate LDA Models

best_model = None
best_coherence_score = -1

for combination in hyperparameter_combinations:
    num_topics, passes, alpha, eta = combination
    lda_model, coherence_score = train_evaluate_lda_model(corpus, dictionary, num_topics, passes, alpha, eta)
    if coherence_score > best_coherence_score:
        best_model = lda_model
        best_coherence_score = coherence_score

print('Best Model:')
print(best_model)
print('Best Coherence Score:', best_coherence_score)

# Comparing over Project Status

In [52]:
# Calculate statistics for each status
status_stats = df.groupby('status').agg({'amount': ['mean', 'median', 'sum']})
status_stats

amount                  
                    mean median        sum
status                                    
abgeschlossen  35.012816   15.0  18101.626
aktiv          29.070664   15.0  52763.255

In [36]:
# Topic model and frequency table by "status"

# Grouping by status and creating topic models
status_groups = df.groupby("status")
for status, group in status_groups:
    # Preprocessing the descriptions
    group["description_tokens"] = group["description"].apply(preprocess_text)

    # Creating the corpus
    dictionary = corpora.Dictionary(group["description_tokens"])
    corpus = [dictionary.doc2bow(tokens) for tokens in group["description_tokens"]

    # Training the LDA model (Latent Dirichlet Allocation)
    num_topics = 10
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Extracting the dominant topics for each document
    group["topic"] = group["description_tokens"].apply(lambda tokens: max(lda_model[dictionary.doc2bow(tokens)], key=lambda x: x[1])[0] + 1)

    # Counting the frequency of topics
    topic_counts = Counter(group["topic"])

    # Outputting the table with the frequency of topics for the status
    print(f"Status: {status}")
    print("Topic\tFrequency")
    print("----------------")
    for topic, count in topic_counts.items():
        print(f"{topic}\t{count}")
    print("\n")

Status: abgeschlossen
Topic	Frequency
----------------
4	30
6	98
10	45
1	31
8	83
2	30
5	65
7	52
3	34
9	49


Status: aktiv
Topic	Frequency
----------------
4	116
10	176
7	328
8	192
9	119
2	113
5	148
1	297
3	141
6	185




In [41]:
# Analyzing top 10 common tokens from each topic

for status, group in status_groups:
    print(f"Status: {status}")
    for topic in range(1, num_topics+1):
        topic_tokens = lda_model.show_topic(topic-1, topn=10)
        tokens = [token for token, _ in topic_tokens]
        print(f"Topic {topic}: {tokens}")
    print("\n")

Status: abgeschlossen
Topic 1: ['mio', 'sowie', 'eur', 'vorhabens', 'vorhaben', 'zielgruppe', 'bevölkerung', 'zugang', 'beitrag', 'phase']
Topic 2: ['mio', 'eur', 'sowie', 'phase', 'rahmen', 'beitrag', 'ziel', 'vorhaben', 'unterstützung', 'bevölkerung']
Topic 3: ['bevölkerung', 'sowie', 'vorhaben', 'mio', 'zielgruppe', 'verbesserung', 'beitrag', 'eur', 'umsetzung', 'finanziert']
Topic 4: ['fonds', 'sowie', 'vorhaben', 'maßnahmen', 'zielgruppe', 'ziel', 'bevölkerung', 'stärkung', 'beitrag', 'sanad']
Topic 5: ['vorhaben', 'sowie', 'vorhabens', 'flüchtlinge', 'beitrag', 'ziel', 'verbesserung', 'maßnahmen', 'mio', 'zielgruppe']
Topic 6: ['sowie', 'fonds', 'mio', 'vorhaben', 'ziel', 'rahmen', 'sollen', 'eur', 'unternehmen', 'beitrag']
Topic 7: ['mio', 'eur', 'sowie', 'vorhabens', 'bevölkerung', 'vorhaben', 'phase', 'maßnahmen', 'verbesserung', 'sollen']
Topic 8: ['sowie', 'mio', 'vorhaben', 'ziel', 'beitrag', 'eur', 'zielgruppe', 'finanzierung', 'unternehmen', 'entwicklung']
Topic 9: ['sowi

# Issues with the first preprocessing
In the previous cells of the Jupyter notebook, the top 10 common tokens from each topic were analyzed using the LDA model. However, it was observed that many unmeaningful words, as well as some other German stop words such as **_'sowie'_** and **_'wurde'_**, and financial units such as **_'mio'_** and **_'eur'_**, were included in several topics. 

Including these unmeaningful and irrelevant words in the topics can lead to noise and reduce the effectiveness of the topic modeling process. Therefore, it is recommended to remove these words in the preprocessing phase before applying the LDA model.

By removing these unmeaningful words and stop words, the topics generated by the LDA model will likely become more focused and meaningful. This will improve the interpretability and usefulness of the topics for further analysis and decision-making.

To remove these words, a preprocessing step can be added to the code. This step can involve tokenizing the text, removing stop words, and filtering out specific financial units such as 'mio' and 'eur'. By performing this preprocessing, the resulting topics will be more representative of the underlying themes and concepts in the dataset.

Overall, removing unmeaningful words, German stop words, and specific financial words from the topics is crucial to enhance the quality and relevance of the topic modeling results. This will enable more accurate and insightful analysis of the dataset.

# Ideas to improve performance of topic modeling 

When focusing on issue areas and thematic aspects of KfW projects, it is important to remove entity names that may introduce noise and reduce the effectiveness of the topic modeling process. This includes removing regions, countries, development agencies, and abbreviations from the text data.

By removing these entity names, the resulting topics generated by the topic modeling algorithm will likely become more focused and representative of the underlying themes and concepts in the dataset. This will improve the interpretability and usefulness of the topics for further analysis and decision-making.

In addition to removing entity names, there are several other ideas to improve the performance of topic modeling:

1. **Remove unmeaningful words and stop words:** Similar to removing entity names, it is recommended to remove unmeaningful words and stop words from the text data. These words can include common words that do not carry much semantic meaning, such as conjunctions, prepositions, and pronouns. By removing these words, the topics generated by the algorithm will be more meaningful and relevant.

2. **Perform stemming or lemmatization of English text data:** Stemming or lemmatization can be applied to reduce words to their base or root form. This helps in consolidating similar words and reducing the dimensionality of the text data. By performing stemming or lemmatization, the topic modeling algorithm can capture the essence of words more accurately and generate more coherent topics.

3. **Consider using n-grams:** Instead of considering only individual words, n-grams can be used to capture the context and relationships between multiple words. This can provide a more comprehensive understanding of the text data and improve the quality of the generated topics.

4. **Optimize hyperparameters:** Topic modeling algorithms often have hyperparameters that can be tuned to improve performance. For example, the number of topics, alpha and beta parameters in LDA, and regularization parameters in NMF can be optimized to obtain better topic representations.

By implementing these ideas, the performance of topic modeling can be enhanced, leading to more accurate and insightful analysis of the dataset.

# Comparing across regions and sectors


## Comparing across regions

To compare the data across different regions, we can use the `fzRegion` column in the `df` DataFrame. Let's calculate some statistics for each region.

In [48]:
# Calculate statistics for each region
region_stats = df.groupby('fzRegion').agg({'amount': ['mean', 'median', 'sum']})
region_stats

amount                    
                             mean   median        sum
fzRegion                                             
                        14.000000  14.0000     14.000
Afrika Subsahara        19.274955  13.0000  16615.011
Asien/Ozeanien          47.766971  19.4590  19871.060
Europa/Kaukasus         28.625888  15.0000   8416.011
Global                  36.783409  20.0000   3236.940
Lateinamerika           46.368732  22.0970   9876.540
Nordafrika/Naher Osten  28.024714  16.4525  12835.319

In [62]:
# Grouping by region and creating topic models
region_groups = df.groupby("fzRegion")
for region, group in region_groups:
    # Preprocessing the descriptions
    group["description_tokens"] = group["description"].apply(preprocess_text)

    # Creating the corpus
    dictionary = corpora.Dictionary(group["description_tokens"])
    corpus = [dictionary.doc2bow(tokens) for tokens in group["description_tokens"]]

    # Training the LDA model (Latent Dirichlet Allocation)
    num_topics = 5
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Extracting the dominant topics for each document
    group["topic"] = group["description_tokens"].apply(lambda tokens: max(lda_model[dictionary.doc2bow(tokens)], key=lambda x: x[1])[0] + 1)

    # Counting the frequency of topics
    topic_counts = Counter(group["topic"])

    # Outputting the table with the frequency of topics for the region
    print(f"Region: {region}")
    print("Topic\tFrequency")
    print("----------------")
    for topic, count in topic_counts.items():
        print(f"{topic}\t{count}")
    print("\n")

Region: 
Topic	Frequency
----------------
4	1


Region: Afrika Subsahara
Topic	Frequency
----------------
1	122
3	249
2	124
5	128
4	239


Region: Asien/Ozeanien
Topic	Frequency
----------------
4	95
5	54
3	80
1	113
2	74


Region: Europa/Kaukasus
Topic	Frequency
----------------
1	48
4	52
5	82
3	66
2	46


Region: Global
Topic	Frequency
----------------
3	13
5	18
2	27
1	18
4	12


Region: Lateinamerika
Topic	Frequency
----------------
1	52
3	73
2	24
4	35
5	29


Region: Nordafrika/Naher Osten
Topic	Frequency
----------------
3	63
2	57
1	80
5	147
4	111




In [63]:
# Analyzing top 10 common words from each topic of each region

for region, region_stats in region_stats.groupby('fzRegion'):
    print(f"Region: {region}")
    for topic in range(1, num_topics+1):
        topic_tokens = lda_model.show_topic(topic-1, topn=10)
        tokens = [token for token, _ in topic_tokens]
        print(f"Topic {topic}: {tokens}")
    print("\n")

Region: Nordafrika/Naher Osten
Topic 1: ['mio', 'eur', 'sowie', 'bevölkerung', 'vorhaben', 'maßnahmen', 'beitrag', 'ziel', 'rahmen', 'phase']
Topic 2: ['eur', 'mio', 'sowie', 'vorhaben', 'bevölkerung', 'phase', 'umfasst', 'sollen', 'verbesserung', 'projektträger']
Topic 3: ['fonds', 'sowie', 'maßnahmen', 'sanad', 'ziel', 'kkmu', 'eu', 'irak', 'finanziert', 'unternehmen']
Topic 4: ['sowie', 'vorhaben', 'maßnahmen', 'ziel', 'vorhabens', 'beitrag', 'mio', 'rahmen', 'verbesserung', 'bevölkerung']
Topic 5: ['flüchtlinge', 'sowie', 'mio', 'libanon', 'maßnahmen', 'verbesserung', 'ziel', 'eur', 'gemeinden', 'jordanien']




## Comparing Across Focus of KFW Projects

To compare across the focus of KFW projects, we can analyze the distribution of project focus areas and identify the most common focus areas. This analysis can provide insights into the areas where KFW projects are predominantly focused.

Let's start by loading the dataset and examining the focus areas.

In [53]:
# Calculate statistics for each region
focus_stats = df.groupby('focus').agg({'amount': ['mean', 'median', 'sum']})
focus_stats

amount                    
                                                mean   median        sum
focus                                                                   
Bildung                                    19.704839   15.000   3310.413
Demokratie                                 16.608856   12.000   2076.107
Energie                                    51.911678   27.015  15936.885
Ernährung und Landwirtschaft               21.335000   12.000   3264.255
Friedensentwicklung und Krisenprävention   23.568911   15.000   3983.146
Gesundheit                                 17.902017   12.000   4171.170
Regionale Konzentration                   127.209143  101.500   3561.856
Sonstige                                   32.334753   14.476   5238.230
Transport und Kommunikation                54.042944   20.000   2918.319
Umwelt und Klima                           17.840885   11.100   4513.744
Wasser und Abfall                          26.599828   18.000   7261.753
Wirtschaft und Finanzsektor                35.943496   14.861  14629.003

In [64]:
# Topic model and frequency table by "focus"

# Grouping by focus and creating topic models
focus_groups = df.groupby("focus")
for focus, group in focus_groups:
    # Preprocessing the descriptions
    group["description_tokens"] = group["description"].apply(preprocess_text)

    # Creating the corpus
    dictionary = corpora.Dictionary(group["description_tokens"])
    corpus = [dictionary.doc2bow(tokens) for tokens in group["description_tokens"]]

    # Training the LDA model (Latent Dirichlet Allocation)
    num_topics = 5
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Extracting the dominant topics for each document
    group["topic"] = group["description_tokens"].apply(lambda tokens: max(lda_model[dictionary.doc2bow(tokens)], key=lambda x: x[1])[0] + 1)

    # Counting the frequency of topics
    topic_counts = Counter(group["topic"])

    # Outputting the table with the frequency of topics for the focus
    print(f"Focus: {focus}")
    print("Topic\tFrequency")
    print("----------------")
    for topic, count in topic_counts.items():
        print(f"{topic}\t{count}")
    print("\n")

Focus: Bildung
Topic	Frequency
----------------
2	37
5	39
3	23
4	31
1	38


Focus: Demokratie
Topic	Frequency
----------------
3	22
5	26
1	25
4	28
2	24


Focus: Energie
Topic	Frequency
----------------
2	64
4	74
5	99
3	45
1	25


Focus: Ernährung und Landwirtschaft
Topic	Frequency
----------------
1	18
2	35
5	18
3	36
4	46


Focus: Friedensentwicklung und Krisenprävention
Topic	Frequency
----------------
4	26
5	50
2	38
3	39
1	16


Focus: Gesundheit
Topic	Frequency
----------------
3	40
4	58
1	87
5	20
2	28


Focus: Regionale Konzentration
Topic	Frequency
----------------
1	6
5	3
2	5
4	13
3	1


Focus: Sonstige
Topic	Frequency
----------------
4	55
2	35
5	20
1	31
3	21


Focus: Transport und Kommunikation
Topic	Frequency
----------------
3	17
1	10
5	4
4	13
2	10


Focus: Umwelt und Klima
Topic	Frequency
----------------
3	68
5	37
2	68
1	42
4	38


Focus: Wasser und Abfall
Topic	Frequency
----------------
5	101
2	52
3	51
1	57
4	12


Focus: Wirtschaft und Finanzsektor
Topic	Frequency
------------

In [65]:
# Analyzing top 10 common words from each topic of each focus area

for focus, focus_stats in focus_stats.groupby('focus'):
    print(f"Focus: {focus}")
    for topic in range(1, num_topics+1):
        topic_tokens = lda_model.show_topic(topic-1, topn=10)
        tokens = [token for token, _ in topic_tokens]
        print(f"Topic {topic}: {tokens}")
    print("\n")

Focus: Bildung
Topic 1: ['fonds', 'sowie', 'unternehmen', 'ziel', 'verbessern', 'sanad', 'entwicklung', 'beitrag', 'ländern', 'mittel']
Topic 2: ['kkmu', 'sowie', 'unternehmen', 'zugang', 'mio', 'eur', 'zielgruppe', 'vorhaben', 'finanzdienstleistungen', 'ziel']
Topic 3: ['kkmu', 'sowie', 'mio', 'unternehmen', 'vorhaben', 'beitrag', 'ziel', 'eur', 'zielgruppe', 'sollen']
Topic 4: ['mio', 'fonds', 'sowie', 'eur', 'investitionen', 'finanzierung', 'wurde', 'verfügung', 'kkmu', 'unternehmen']
Topic 5: ['mio', 'ziel', 'unternehmen', 'mef', 'vorhaben', 'eur', 'rahmen', 'vorhabens', 'fazilität', 'beschäftigung']


Focus: Demokratie
Topic 1: ['fonds', 'sowie', 'unternehmen', 'ziel', 'verbessern', 'sanad', 'entwicklung', 'beitrag', 'ländern', 'mittel']
Topic 2: ['kkmu', 'sowie', 'unternehmen', 'zugang', 'mio', 'eur', 'zielgruppe', 'vorhaben', 'finanzdienstleistungen', 'ziel']
Topic 3: ['kkmu', 'sowie', 'mio', 'unternehmen', 'vorhaben', 'beitrag', 'ziel', 'eur', 'zielgruppe', 'sollen']
Topic 4: [

## Compare across crscode2

In [66]:
# Calculate statistics for each crscode2
crscode2_stats = df.groupby('crscode2').agg({'amount': ['mean', 'median', 'sum']})
crscode2_stats

amount  ...           
                                                          mean  ...        sum
crscode2                                                        ...           
000 - Förderbereich nicht definiert                   8.932500  ...     17.865
110 - Bildung                                        19.576247  ...   4365.503
120 - Gesundheitswesen                               23.617194  ...   4133.009
130 - Bevölkerungspolitik/-programme u. repd. G...   10.721521  ...    782.671
140 - Wasser und Abwasser-/Abfallentsorgung          28.326806  ...   7903.179
150 - Staat und Zivilgesellschaft                    31.400173  ...   4710.026
160 - Sonstige soziale Infrastruktur und Dienste     30.209730  ...   3806.426
210 - Transport und Lagerhaltung                     85.311600  ...   4265.580
220 - Kommunikation                                  11.255000  ...     22.510
230 - Energieerzeugung, -verteilung und -effizienz   50.952645  ...  15489.604
240 - Finanzwesen                                    34.149752  ...  10893.771
250 - Privatwirtschaftliche und andere Dienste       19.023556  ...    171.212
310 - Landwirtschaft, Forstwirtschaft und Fisch...   22.723340  ...   3340.331
320 - Industrie. Bodenschätze und Bergbau, Bauw...   35.197760  ...    879.944
330 - Handel und Tourismus                          102.250000  ...    409.000
410 - Umweltschutz allgemein                         17.852115  ...   3873.909
430 - Andere multisektorale Maßnahmen                34.074646  ...   3850.435
510 - Allgemeine Budgethilfe                         36.666667  ...    110.000
520 - Nahrungsmittelhilfe/Hilfe zur Ernährungss...   15.144923  ...    196.884
720 - Soforthilfe und entsprechende Dienstleist...   16.416725  ...    656.669
730 - Wiederaufbauhilfe u. Wiederherstellungsma...   14.056373  ...    716.875
740 - Katastrophenprävention und Vorsorge            15.000000  ...     15.000
930 - Flüchtlingshilfe im Geberland                   8.750000  ...     17.500
990 - Nicht zuordenbare Maßnahmen                    59.244500  ...    236.978

[24 rows x 3 columns]

In [67]:
# Topic model and frequency table by crscode2

# Grouping by crscode2 and creating topic models
crscode2_groups = df.groupby("crscode2")
for crscode2, group in crscode2_groups:
    # Preprocessing the descriptions
    group["description_tokens"] = group["description"].apply(preprocess_text)

    # Creating the corpus
    dictionary = corpora.Dictionary(group["description_tokens"])
    corpus = [dictionary.doc2bow(tokens) for tokens in group["description_tokens"]]

    # Training the LDA model (Latent Dirichlet Allocation)
    num_topics = 5
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Extracting the dominant topics for each document
    group["topic"] = group["description_tokens"].apply(lambda tokens: max(lda_model[dictionary.doc2bow(tokens)], key=lambda x: x[1])[0] + 1)

    # Counting the frequency of topics
    topic_counts = Counter(group["topic"])

    # Outputting the table with the frequency of topics for the crscode2
    print(f"crscode2: {crscode2}")
    print("Topic\tFrequency")
    print("----------------")
    for topic, count in topic_counts.items():
        print(f"{topic}\t{count}")
    print("\n")

crscode2: 000 - Förderbereich nicht definiert
Topic	Frequency
----------------
3	1
4	1


crscode2: 110 - Bildung
Topic	Frequency
----------------
4	65
2	39
1	44
3	28
5	47


crscode2: 120 - Gesundheitswesen
Topic	Frequency
----------------
5	32
4	52
3	30
2	36
1	25


crscode2: 130 - Bevölkerungspolitik/-programme u. repd. Gesundheit
Topic	Frequency
----------------
5	26
4	20
1	15
2	4
3	8


crscode2: 140 - Wasser und Abwasser-/Abfallentsorgung
Topic	Frequency
----------------
1	91
5	72
3	53
4	25
2	38


crscode2: 150 - Staat und Zivilgesellschaft
Topic	Frequency
----------------
2	26
3	44
5	43
4	19
1	18


crscode2: 160 - Sonstige soziale Infrastruktur und Dienste
Topic	Frequency
----------------
4	39
3	19
1	33
2	6
5	29


crscode2: 210 - Transport und Lagerhaltung
Topic	Frequency
----------------
2	6
5	13
1	9
4	14
3	8


crscode2: 220 - Kommunikation
Topic	Frequency
----------------
5	2


crscode2: 230 - Energieerzeugung, -verteilung und -effizienz
Topic	Frequency
----------------
4	124
1	60

In [68]:
# Analyzing top 10 common words from each topic of each crscode2

for crscode2, crscode2_stats in crscode2_stats.groupby('crscode2'):
    print(f"crscode2: {crscode2}")
    for topic in range(1, num_topics+1):
        topic_tokens = lda_model.show_topic(topic-1, topn=10)
        tokens = [token for token, _ in topic_tokens]
        print(f"Topic {topic}: {tokens}")
    print("\n")

crscode2: 000 - Förderbereich nicht definiert
Topic 1: ['partech', 'africa', 'afrika', 'tcx', 'dabei', 'ee', 'fund', 'fonds', 'verbesserung', 'beitrag']
Topic 2: ['mio', 'kfw', 'eur', 'darlehen', 'usd', 'projekt', 'finanzierung', 'erste', 'development', 'entwicklung']
Topic 3: ['energieversorgung', 'herausforderungen', 'ee', 'tcx', 'afrika', 'finanzierung', 'beitrag', 'geleistet', 'zugang', 'verbesserung']
Topic 4: ['partech', 'africa', 'fonds', 'fund', 'eur', 'mio', 'afrika', 'beitrag', 'finanzierung', 'seit']
Topic 5: ['agf', 'sdg', 'afrika', 'mio', 'eur', 'kkmu', 'vergeben', 'kkmus', 'guarantee', 'dadurch']


crscode2: 110 - Bildung
Topic 1: ['partech', 'africa', 'afrika', 'tcx', 'dabei', 'ee', 'fund', 'fonds', 'verbesserung', 'beitrag']
Topic 2: ['mio', 'kfw', 'eur', 'darlehen', 'usd', 'projekt', 'finanzierung', 'erste', 'development', 'entwicklung']
Topic 3: ['energieversorgung', 'herausforderungen', 'ee', 'tcx', 'afrika', 'finanzierung', 'beitrag', 'geleistet', 'zugang', 'verbess